Primero, preprocesamiento para ver cuantas variables se van a meter a la red

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Actual con diferencias, previo no, para hacer dos GRU

In [2]:
actual_diferencias=pd.read_csv('../data/actual_diferencias_preproc_escalado.csv')
previos=pd.read_csv('../data/previos_preproc_escalado.csv')

In [3]:
print(actual_diferencias.shape, previos.shape)

(9259, 19) (925900, 33)


In [4]:
print(previos.shape[0]/actual_diferencias.shape[0])
num_previos=int(previos.shape[0]/actual_diferencias.shape[0])
print(num_previos)

100.0
100


## RED NEURONAL PARA PREDECIR LA PROBABILIDAD DE GANAR SEGUN EL MERCADO

Primero valorar cuantas filas se pierden por predecir esto

In [7]:
actual_diferencias[['probability_home', 'probability_away', 'vigorish']].describe()

,probability_home,probability_away,vigorish
count,8466.000000,8466.000000,8466.000000
mean,0.511669,0.488331,0.057569
std,0.206690,0.206690,0.011553
min,0.023902,0.023902,0.020406
25%,0.354949,0.337349,0.050000
50%,0.514706,0.485294,0.054945
75%,0.662651,0.645051,0.063348
max,0.976098,0.976098,0.108140


Merece la pena eliminar 793 filas de 9259, es un porcentaje muy bajo. Además serán torneos menores (rondas clasificatorias y asi)

In [8]:
actual_diferencias[['probability_home', 'probability_away', 'vigorish']].isnull().sum()

probability_home    793
probability_away    793
vigorish            793
dtype: int64

In [9]:
actual_diferencias.shape

(9259, 19)

In [10]:
filas_nulas = actual_diferencias[actual_diferencias.isnull().any(axis=1)].index.tolist()
indices_previos_eliminar = []
for idx in filas_nulas:
    inicio = idx * num_previos
    fin = inicio + num_previos  # no es inclusivo
    indices_previos_eliminar.extend(range(inicio, fin))
actual_diferencias.drop(index=filas_nulas, inplace=True)
previos.drop(index=indices_previos_eliminar, inplace=True)
print(actual_diferencias.shape, previos.shape)

(8466, 19) (846600, 33)


In [11]:
vigorish=actual_diferencias['vigorish']
actual_diferencias=actual_diferencias.drop(columns=['vigorish', 'winnerCode', 'probability_home'])
actual_diferencias.head()

,porcentajeActualRanking,porcentajeBestRanking,difHeight,difWeight,probability_away,groundType_Grass,groundType_Hardcourt_indoor,groundType_Hardcourt_outdoor,groundType_Red_clay,difLaterality_00,difLaterality_01,difLaterality_10,difLaterality_11,best_five,difYear,year_week_id
0,-0.096934,-0.520517,-1.044343,0.228463,0.766284,0,0,1,0,0,0,1,0,0,0.859275,0
1,0.177778,-0.246956,0.738353,0.228463,0.392308,0,0,1,0,0,0,0,1,0,-0.943432,0
2,-0.087356,0.639723,0.528624,0.561874,0.441489,0,0,1,0,0,0,0,1,0,0.801265,0
3,0.198134,1.104192,-0.205428,0.228463,0.135802,0,0,1,0,0,0,0,1,0,0.091960,0
4,-0.153211,-0.777271,0.528624,1.006422,0.422572,0,0,1,0,0,0,0,1,0,-0.290819,0


In [12]:
num_filas = len(previos)
tamano_bloque=int(num_previos/2)
# Crear listas para almacenar los índices
indices_home = []
indices_away = []

# Para cada bloque, extraer los índices correspondientes
for i in range(actual_diferencias.shape[0]):
    inicio_bloque = i * tamano_bloque * 2
    
    # Índices para home (primeras 50 filas del bloque)
    indices_home.extend(range(inicio_bloque, inicio_bloque + tamano_bloque))
    
    # Índices para away (siguientes 50 filas del bloque)
    indices_away.extend(range(inicio_bloque + tamano_bloque, inicio_bloque + tamano_bloque * 2))
    
# Crear DataFrames usando los índices
previos_home = previos.iloc[indices_home].reset_index(drop=True)
previos_away = previos.iloc[indices_away].reset_index(drop=True)
actual_diferencias=actual_diferencias.reset_index(drop=True)
print(actual_diferencias.shape, previos_home.shape, previos_away.shape)
print(actual_diferencias.isnull().sum().sum(), previos_home.isnull().sum().sum(), previos_away.isnull().sum().sum())

(8466, 16) (423300, 33) (423300, 33)
0 0 0


In [13]:
actual_diferencias.head(5)

,porcentajeActualRanking,porcentajeBestRanking,difHeight,difWeight,probability_away,groundType_Grass,groundType_Hardcourt_indoor,groundType_Hardcourt_outdoor,groundType_Red_clay,difLaterality_00,difLaterality_01,difLaterality_10,difLaterality_11,best_five,difYear,year_week_id
0,-0.096934,-0.520517,-1.044343,0.228463,0.766284,0,0,1,0,0,0,1,0,0,0.859275,0
1,0.177778,-0.246956,0.738353,0.228463,0.392308,0,0,1,0,0,0,0,1,0,-0.943432,0
2,-0.087356,0.639723,0.528624,0.561874,0.441489,0,0,1,0,0,0,0,1,0,0.801265,0
3,0.198134,1.104192,-0.205428,0.228463,0.135802,0,0,1,0,0,0,0,1,0,0.091960,0
4,-0.153211,-0.777271,0.528624,1.006422,0.422572,0,0,1,0,0,0,0,1,0,-0.290819,0


#### RED

semanas 0-37: 2021, 38-79: 2022, 80-122: 2023, 123-164: 2024

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import copy
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

ejemplos_predicciones = []

class TennisRNN(nn.Module):
    def __init__(self, input_size_actual, input_size_previos, hidden_size, num_layers, output_size, dropout_rate):
        super(TennisRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # GRU para datos históricos del jugador local
        self.gru_home = nn.GRU(
            input_size=input_size_previos,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0
        )
        # GRU para datos históricos del jugador visitante
        self.gru_away = nn.GRU(
            input_size=input_size_previos,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0
        )
        # Capa de proyección para las características actuales
        self.actual_projection = nn.Linear(input_size_actual, input_size_actual * 2)

        # Capa de concatenación y fully connected
        combined_size = hidden_size * 2 + input_size_actual * 2  
        self.fc1 = nn.Linear(combined_size, hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.fc2 = nn.Linear(hidden_size, hidden_size//2)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.fc3 = nn.Linear(hidden_size//2, output_size)
        
    def forward(self, x_actual, x_home, x_away):
        _, (home_features, _) = self.gru_home(x_home)
        _, (away_features, _) = self.gru_away(x_away)
        
        # Capa de proyección para las características actuales
        x_actual_projected = self.actual_projection(x_actual)
        combined = torch.cat((home_features, away_features, x_actual_projected), dim=1)

        # Pasar por las capas fully connected
        x = torch.relu(self.fc1(combined))
        x = self.dropout1(x)

        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)

        # Utilizamos sigmoid para obtener un valor entre 0 y 1 (probabilidad)
        x = torch.sigmoid(x)
        
        return x
    
def calculate_metrics(pred, target):
    """
    Calcula métricas específicas para regresión
    """
    pred_np = pred.cpu().numpy().flatten()
    target_np = target.cpu().numpy().flatten()
    
    # Calcular métricas
    mae = mean_absolute_error(target_np, pred_np)
    mse = mean_squared_error(target_np, pred_np)
    rmse = np.sqrt(mse)
    r2 = r2_score(target_np, pred_np)
    
    # MAPE (Mean Absolute Percentage Error) - cuidado con divisiones por cero
    mape = np.mean(np.abs((target_np - pred_np) / np.clip(target_np, 1e-8, None))) * 100
    
    return {
        'mae': mae,
        'mse': mse,
        'rmse': rmse,
        'r2': r2,
        'mape': mape
    }

def guardar_ejemplos_prediccion(y_pred, y_true, indices_test, ventana, umbral_precision=0.05, umbral_error=0.20):
    """
    Guarda ejemplos de predicciones para análisis posterior
    """
    y_pred_np = y_pred.cpu().numpy().flatten()
    y_true_np = y_true.cpu().numpy().flatten()
    
    # Calcular errores absolutos
    errores = np.abs(y_pred_np - y_true_np)
    
    # Casos de alta precisión (error < umbral_precision)
    indices_precision = np.where(errores < umbral_precision)[0]
    
    # Casos de error significativo (error > umbral_error)
    indices_error = np.where(errores > umbral_error)[0]
    
    # Guardar algunos ejemplos de cada tipo (máximo 3 por ventana)
    for idx in indices_precision[:3]:
        ejemplo = {
            'ventana': ventana,
            'tipo': 'alta_precision',
            'indice_dataset': indices_test[idx],
            'prediccion': y_pred_np[idx],
            'real': y_true_np[idx],
            'error': errores[idx]
        }
        ejemplos_predicciones.append(ejemplo)
    
    for idx in indices_error[:3]:
        ejemplo = {
            'ventana': ventana,
            'tipo': 'error_significativo',
            'indice_dataset': indices_test[idx],
            'prediccion': y_pred_np[idx],
            'real': y_true_np[idx],
            'error': errores[idx]
        }
        ejemplos_predicciones.append(ejemplo)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
total_partidos = len(actual_diferencias)
num_previos = 50  
input_size_actual = actual_diferencias.drop(columns=['probability_away', 'year_week_id']).shape[1]
input_size_previos = previos.shape[1]

df_visualization_global = pd.DataFrame({
    'Window': pd.Series(dtype='int'),        
    'Epoch': pd.Series(dtype='int'),      
    'TrainingLoss': pd.Series(dtype='float'), 
    'ValidationLoss': pd.Series(dtype='float'),
    'MAE': pd.Series(dtype='float'),
    'MSE': pd.Series(dtype='float'), 
    'RMSE': pd.Series(dtype='float'),
    'R2': pd.Series(dtype='float'),
    'MAPE': pd.Series(dtype='float'),
})

# Inicialización del modelo
model = TennisRNN(
    input_size_actual=input_size_actual,
    input_size_previos=input_size_previos,
    hidden_size=128,
    num_layers=2,
    output_size=1,
    dropout_rate=0.2,
).to(device)


criterion = nn.MSELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Inicializar el estado del modelo para la primera ventana
# Parámetros para early stopping
previous_best_model_state = None
patience = 5  # Número de épocas a esperar antes de detener el entrenamiento si no hay mejora
min_delta = 0.00001  # Mejora mínima requerida para considerar que hay progreso

# Parámetros de entrenamiento
epochs = 100
view_step = 5

training_weeks = 10  # Número de semanas para entrenamiento
testing_weeks = 3    # Número de semanas para prueba
step_size = 1        # Paso de avance entre ventanas (en semanas)
all_year_weeks = actual_diferencias['year_week_id'].unique()

total_windows = (len(all_year_weeks) - (training_weeks + testing_weeks)) // step_size + 1
# Bucle de entrenamiento con ventanas de tiempo
for i in range(0, len(all_year_weeks) - (training_weeks + testing_weeks) + 1, step_size):
    # Índice de la ventana actual (comenzando desde 1)
    current_window = i // step_size + 1
    print(f"Ventana {current_window} de {total_windows}")

    # Semanas para entrenamiento y prueba
    train_year_weeks = all_year_weeks[i:i+training_weeks]
    test_year_weeks = all_year_weeks[i+training_weeks:i+training_weeks+testing_weeks]
   
    train_indices = actual_diferencias[actual_diferencias['year_week_id'].isin(train_year_weeks)].index
    test_indices = actual_diferencias[actual_diferencias['year_week_id'].isin(test_year_weeks)].index
    
    print(f"Entrenando con semanas: {train_year_weeks}, testeando en semanas {test_year_weeks}, len train: {len(train_indices)}, len test: {len(test_indices)}, porcentaje: {len(train_indices)/(len(train_indices)+len(test_indices)):.2f}")

    # Comprobar si hay suficientes datos
    if len(train_indices) < 50 or len(test_indices) < 10:
        print(f"Saltando ventana {current_window} por datos insuficientes: train={len(train_indices)}, test={len(test_indices)}")
        continue
    
    # Preparar los datos de entrenamiento y test
    X_train_actual = actual_diferencias.iloc[train_indices].drop(columns=['probability_away', 'year_week_id'])
    Y_train_actual = actual_diferencias.iloc[train_indices]['probability_away'].values.reshape(-1, 1)
    
    X_test_actual = actual_diferencias.iloc[test_indices].drop(columns=['probability_away', 'year_week_id'])
    Y_test_actual = actual_diferencias.iloc[test_indices]['probability_away'].values.reshape(-1, 1)
    
    previos_train_index= []
    for idx in train_indices:
        inicio = idx * num_previos
        fin = inicio + num_previos 
        previos_train_index.extend(range(inicio, fin))
    
    previos_test_index= []
    for idx in test_indices:
        inicio = idx * num_previos
        fin = inicio + num_previos  
        previos_test_index.extend(range(inicio, fin))
    
    previos_train_home = previos_home.iloc[previos_train_index].values
    previos_train_away = previos_away.iloc[previos_train_index].values
    previos_test_home = previos_home.iloc[previos_test_index].values
    previos_test_away = previos_away.iloc[previos_test_index].values

    # Reshape para GRU: (n_samples, seq_length, n_features)
    n_train_samples= len(train_indices)
    n_test_samples= len(test_indices)
    seq_length = num_previos
    n_features = input_size_previos

    X_train_previos_home = previos_train_home.reshape(n_train_samples, seq_length, n_features)
    X_train_previos_away = previos_train_away.reshape(n_train_samples, seq_length, n_features)
    X_test_previos_home = previos_test_home.reshape(n_test_samples, seq_length, n_features)
    X_test_previos_away = previos_test_away.reshape(n_test_samples, seq_length, n_features)

    # Convertir a tensores
    tensor_X_train_actual = torch.tensor(X_train_actual.values, dtype=torch.float32).to(device)
    tensor_Y_train_actual = torch.tensor(Y_train_actual, dtype=torch.float32).to(device)
    tensor_X_test_actual = torch.tensor(X_test_actual.values, dtype=torch.float32).to(device)
    tensor_Y_test_actual = torch.tensor(Y_test_actual, dtype=torch.float32).to(device)

    tensor_X_train_previos_home = torch.tensor(X_train_previos_home, dtype=torch.float32).to(device)
    tensor_X_train_previos_away = torch.tensor(X_train_previos_away, dtype=torch.float32).to(device)
    tensor_X_test_previos_home = torch.tensor(X_test_previos_home, dtype=torch.float32).to(device)
    tensor_X_test_previos_away = torch.tensor(X_test_previos_away, dtype=torch.float32).to(device)

    # Early stopping
    best_val_loss = float('inf')
    best_model_state = None
    best_predictions = None
    best_true_values = None
    patience_counter = 0
    epoca_mejor = 0

    # Aplicar warm-up si no es la primera ventana
    if i > 0 and previous_best_model_state is not None:
        model.load_state_dict(previous_best_model_state)
        print(f"Iniciando ventana {current_window} con pesos de la ventana anterior")
    
    # Entrenamiento
    for epoch in range(epochs):
        model.train()
        
        # Forward pass
        y_pred = model(tensor_X_train_actual, tensor_X_train_previos_home, tensor_X_train_previos_away)
        loss = criterion(y_pred, tensor_Y_train_actual)
        
        # Backward pass y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss = loss.item()

        
        model.eval()
        with torch.no_grad():
            y_test_pred = model(tensor_X_test_actual, tensor_X_test_previos_home, tensor_X_test_previos_away)
            test_loss = criterion(y_test_pred, tensor_Y_test_actual).item()

            # Calcular métricas de clasificación
            metrics = calculate_metrics(y_test_pred, tensor_Y_test_actual)

            # Guardar ejemplos de predicción
            if epoch == 0 or (best_val_loss - test_loss > min_delta):
                guardar_ejemplos_prediccion(y_test_pred, tensor_Y_test_actual, test_indices, current_window)
            # Early stopping
            if best_val_loss - test_loss > min_delta:
                best_val_loss = test_loss
                # Guardar el estado del modelo
                best_model_state = copy.deepcopy(model.state_dict())
                patience_counter = 0
                epoca_mejor = epoch
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f'Ventana {current_window}, Época: {epoch}, Train Loss: {train_loss:.8f}, Test Loss: {test_loss:.8f}, R2: {metrics["r2"]:.4f}, MAE: {metrics["mae"]:.4f}')
                # Guardar información para visualización
                new_row = pd.DataFrame({
                    'Window': [current_window],
                    'Epoch': [epoch],
                    'TrainingLoss': [train_loss],
                    'ValidationLoss': [test_loss],
                    'MAE': [metrics["mae"]],
                    'MSE': [metrics["mse"]],
                    'RMSE': [metrics["rmse"]],
                    'R2': [metrics["r2"]],
                    'MAPE': [metrics["mape"]],
                })
                df_visualization_global = pd.concat([df_visualization_global, new_row], ignore_index=True)
                print(f"Early stopping en la ventana {current_window}, época {epoch}")
                break
                

        if epoch % view_step == 0:
            print(f'Ventana {current_window}, Época: {epoch}, Train Loss: {train_loss:.8f}, Test Loss: {test_loss:.8f},R2: {metrics["r2"]:.4f}, MAE: {metrics["mae"]:.4f}')
            


        # Guardar información para visualización
        new_row = pd.DataFrame({
                    'Window': [current_window],
                    'Epoch': [epoch],
                    'TrainingLoss': [train_loss],
                    'ValidationLoss': [test_loss],
                    'MAE': [metrics["mae"]],
                    'MSE': [metrics["mse"]],
                    'RMSE': [metrics["rmse"]],
                    'R2': [metrics["r2"]],
                    'MAPE': [metrics["mape"]],
        })

        df_visualization_global = pd.concat([df_visualization_global, new_row], ignore_index=True)
       
    # Cargar el mejor modelo
    if best_model_state is not None:
        model.load_state_dict(best_model_state)  # Cargar el mejor modelo de esta ventana
        previous_best_model_state = copy.deepcopy(best_model_state)  # Guardar para la siguiente ventana
        print(f"Mejor modelo guardado en la época  {epoca_mejor} para la ventana {current_window}")

    print("-------------------") 

# Guardar el modelo
torch.save(model.state_dict(), 'tennis_rnn_model_REGRESSION.pt')

Ventana 1 de 143
Entrenando con semanas: [0 1 2 3 4 5 6 7 8 9], testeando en semanas [10 11 12], len train: 506, len test: 140, porcentaje: 0.78
Ventana 1, Época: 0, Train Loss: 0.04670256, Test Loss: 0.03795493,R2: -0.0132, MAE: 0.1547


C:\Users\diego\AppData\Local\Temp\ipykernel_19644\691616427.py:289: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_visualization_global = pd.concat([df_visualization_global, new_row], ignore_index=True)


Ventana 1, Época: 5, Train Loss: 0.04027548, Test Loss: 0.03677423,R2: 0.0183, MAE: 0.1522
Ventana 1, Época: 10, Train Loss: 0.03228220, Test Loss: 0.03839981,R2: -0.0251, MAE: 0.1557
Ventana 1, Época: 12, Train Loss: 0.03071227, Test Loss: 0.03869670, R2: -0.0330, MAE: 0.1564
Early stopping en la ventana 1, época 12
Mejor modelo guardado en la época  7 para la ventana 1
-------------------
Ventana 2 de 143
Entrenando con semanas: [ 1  2  3  4  5  6  7  8  9 10], testeando en semanas [11 12 13], len train: 498, len test: 121, porcentaje: 0.80
Iniciando ventana 2 con pesos de la ventana anterior
Ventana 2, Época: 0, Train Loss: 0.03661659, Test Loss: 0.03736274,R2: 0.0216, MAE: 0.1532
Ventana 2, Época: 5, Train Loss: 0.02989524, Test Loss: 0.03652114,R2: 0.0436, MAE: 0.1524
Ventana 2, Época: 9, Train Loss: 0.02387996, Test Loss: 0.03938857, R2: -0.0315, MAE: 0.1587
Early stopping en la ventana 2, época 9
Mejor modelo guardado en la época  4 para la ventana 2
-------------------
Ventana 

In [ ]:
df_ejemplos = pd.DataFrame(ejemplos_predicciones)
df_ejemplos.to_csv('ejemplos_predicciones_regresion.csv', index=False)
df_visualization_global.to_csv('resultados/df_visualization_global_REGRESSION.csv', index=False)

# RED NEURONAL PARA PREDEICR EL WINNERCODE

In [17]:
%reset -f

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Actual con diferencias, previo no, para hacer dos gru

In [19]:
actual_diferencias=pd.read_csv('../data/actual_diferencias_preproc_escalado.csv')
actual_diferencias=actual_diferencias.drop(columns=['probability_home', 'probability_away', 'vigorish'])
previos=pd.read_csv('../data/previos_preproc_escalado.csv')

In [20]:
print(actual_diferencias.shape, previos.shape)

(9259, 16) (925900, 33)


In [21]:
print(previos.shape[0]/actual_diferencias.shape[0])
num_previos=int(previos.shape[0]/actual_diferencias.shape[0])
print(num_previos)

100.0
100


In [22]:
actual_diferencias.head()

,winnerCode,porcentajeActualRanking,porcentajeBestRanking,difHeight,difWeight,groundType_Grass,groundType_Hardcourt_indoor,groundType_Hardcourt_outdoor,groundType_Red_clay,difLaterality_00,difLaterality_01,difLaterality_10,difLaterality_11,best_five,difYear,year_week_id
0,1,-0.096934,-0.520517,-1.044343,0.228463,0,0,1,0,0,0,1,0,0,0.859275,0
1,0,0.177778,-0.246956,0.738353,0.228463,0,0,1,0,0,0,0,1,0,-0.943432,0
2,1,-0.087356,0.639723,0.528624,0.561874,0,0,1,0,0,0,0,1,0,0.801265,0
3,0,0.198134,1.104192,-0.205428,0.228463,0,0,1,0,0,0,0,1,0,0.091960,0
4,0,-0.153211,-0.777271,0.528624,1.006422,0,0,1,0,0,0,0,1,0,-0.290819,0


In [23]:
previos.head()

,winnerCode,ActualRankingHome,BestRankingHome,HeightHome,WeightHome,RightHandedHome,ActualRankingAway,BestRankingAway,HeightAway,WeightAway,RightHandedAway,homeScore,awayScore,set1performanceHome,set1performanceAway,set2performanceHome,set2performanceAway,set3performanceHome,set3performanceAway,set4performanceHome,set4performanceAway,set5performanceHome,set5performanceAway,totalGamesHome,totalGamesAway,days_no_played,best_five,edad_home,edad_away,groundType_Grass,groundType_Hardcourt_indoor,groundType_Hardcourt_outdoor,groundType_Red_clay
0,1,0.817778,0.876667,-1.264435,0.031102,1,0.574444,0.946667,-0.248839,1.110974,1,0.666667,0.000000,0.6,0.7,0.5,0.7,0.00,0.000000,0.0,0.0,0.0,0.0,0.275,0.35,2.049115,1,0.661493,1.198959,1,0,0,0
1,1,0.821111,0.876667,-1.264435,0.031102,1,0.724444,0.775556,-0.540683,-0.289252,1,0.666667,0.333333,0.7,0.5,0.5,0.7,0.25,0.428571,0.0,0.0,0.0,0.0,0.400,0.45,0.064228,0,0.665848,-0.414577,0,0,0,1
2,0,0.821111,0.876667,-1.264435,0.031102,1,0.645556,0.688889,-1.270293,-0.133672,0,0.666667,0.000000,0.7,0.6,0.6,0.4,0.00,0.000000,0.0,0.0,0.0,0.0,0.325,0.25,0.201117,0,0.670825,0.899011,0,0,0,1
3,1,0.821111,0.876667,-1.264435,0.031102,1,0.557778,0.695556,-0.978449,-1.067156,0,0.000000,0.666667,0.2,0.6,0.4,0.6,0.00,0.000000,0.0,0.0,0.0,0.0,0.150,0.30,-0.277993,0,0.671447,-1.319369,0,0,0,1
4,0,0.821111,0.876667,-1.264435,0.031102,1,0.504444,0.635556,-1.270293,-0.289252,0,0.666667,0.000000,0.7,0.5,0.7,0.5,0.00,0.000000,0.0,0.0,0.0,0.0,0.350,0.25,-0.141105,0,0.673935,0.228611,0,0,0,1


In [24]:
num_filas = len(previos)
tamano_bloque=int(num_previos/2)
# Crear listas para almacenar los índices
indices_home = []
indices_away = []

# Para cada bloque, extraer los índices correspondientes
print(actual_diferencias.shape[0])
for i in range(actual_diferencias.shape[0]):
    inicio_bloque = i * tamano_bloque * 2
    
    # Índices para home (primeras 50 filas del bloque)
    indices_home.extend(range(inicio_bloque, inicio_bloque + tamano_bloque))
    
    # Índices para away (siguientes 50 filas del bloque)
    indices_away.extend(range(inicio_bloque + tamano_bloque, inicio_bloque + tamano_bloque * 2))
    
# Crear DataFrames usando los índices
previos_home = previos.iloc[indices_home].reset_index(drop=True)
previos_away = previos.iloc[indices_away].reset_index(drop=True)
actual_diferencias=actual_diferencias.reset_index(drop=True)
print(actual_diferencias.shape, previos_home.shape, previos_away.shape)

9259
(9259, 16) (462950, 33) (462950, 33)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import copy
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

class TennisRNN(nn.Module):
    def __init__(self, input_size_actual, input_size_previos, hidden_size, num_layers, output_size, dropout_rate):
        super(TennisRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # GRU para datos históricos del jugador local
        self.gru_home = nn.GRU(
            input_size=input_size_previos,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0
        )
        # GRU para datos históricos del jugador visitante
        self.gru_away = nn.GRU(
            input_size=input_size_previos,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0
        )
        # Capa de proyección para las características actuales
        self.actual_projection = nn.Linear(input_size_actual, input_size_actual * 2)

        # Capa de concatenación y fully connected
        combined_size = hidden_size * 2 + input_size_actual * 2  
        self.fc1 = nn.Linear(combined_size, hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.fc2 = nn.Linear(hidden_size, hidden_size//2)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.fc3 = nn.Linear(hidden_size//2, output_size)
        
    def forward(self, x_actual, x_home, x_away):
        _, (home_features, _) = self.gru_home(x_home)
        _, (away_features, _) = self.gru_away(x_away)

        # Proyección adicional para las características actuales
        x_actual_projection = self.actual_projection(x_actual)
        combined = torch.cat((home_features, away_features, x_actual_projection), dim=1)

        # Pasar por las capas fully connected
        x = torch.relu(self.fc1(combined))
        x = self.dropout1(x)

        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)

        # Utilizamos sigmoid para obtener un valor entre 0 y 1 (probabilidad)
        x = torch.sigmoid(x)
        
        return x
    
# Cálculo del R2 para tener una medida de bondad de ajuste
def calculate_metrics(pred, target, threshold=0.5):
    # Convertir probabilidades a clases binarias
    pred_class = (pred >= threshold).float().cpu().numpy()
    target_class = target.cpu().numpy()
    
    # Calcular métricas
    accuracy = accuracy_score(target_class, pred_class)
    precision = precision_score(target_class, pred_class, zero_division=0)
    recall = recall_score(target_class, pred_class, zero_division=0)
    f1 = f1_score(target_class, pred_class, zero_division=0)
    auc = roc_auc_score(target_class, pred.cpu().numpy())
 
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc
    }




device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
total_partidos = len(actual_diferencias)
num_previos = 50  
input_size_actual = actual_diferencias.drop(columns=['winnerCode', 'year_week_id']).shape[1]
input_size_previos = previos.shape[1]

df_visualization_global = pd.DataFrame({
    'Window': pd.Series(dtype='int'),
    'Epoch': pd.Series(dtype='int'),
    'TrainingLoss': pd.Series(dtype='float'),
    'ValidationLoss': pd.Series(dtype='float'),
    'Accuracy': pd.Series(dtype='float'),
    'Precision': pd.Series(dtype='float'),
    'Recall': pd.Series(dtype='float'),
    'F1': pd.Series(dtype='float'),
    'AUC': pd.Series(dtype='float'),
})


# Inicialización del modelo
model = TennisRNN(
    input_size_actual=input_size_actual,
    input_size_previos=input_size_previos,
    hidden_size=128,
    num_layers=2,
    output_size=1,
    dropout_rate=0.2,
).to(device)


criterion = nn.BCELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Inicializar el estado del modelo para la primera ventana
# Parámetros para early stopping
previous_best_model_state = None
patience = 5  # Número de épocas a esperar antes de detener el entrenamiento si no hay mejora
min_delta = 0.00001  # Mejora mínima requerida para considerar que hay progreso

# Parámetros de entrenamiento
epochs = 100
view_step = 5

training_weeks = 10  # Número de semanas para entrenamiento
testing_weeks = 3    # Número de semanas para prueba
step_size = 1        # Paso de avance entre ventanas (en semanas)
all_year_weeks = actual_diferencias['year_week_id'].unique()

total_windows = (len(all_year_weeks) - (training_weeks + testing_weeks)) // step_size + 1
# Bucle de entrenamiento con ventanas de tiempo
for i in range(0, len(all_year_weeks) - (training_weeks + testing_weeks) + 1, step_size):
    # Índice de la ventana actual (comenzando desde 1)
    current_window = i // step_size + 1
    print(f"Ventana {current_window} de {total_windows}")

    # Semanas para entrenamiento y prueba
    train_year_weeks = all_year_weeks[i:i+training_weeks]
    test_year_weeks = all_year_weeks[i+training_weeks:i+training_weeks+testing_weeks]
   
    train_indices = actual_diferencias[actual_diferencias['year_week_id'].isin(train_year_weeks)].index
    test_indices = actual_diferencias[actual_diferencias['year_week_id'].isin(test_year_weeks)].index
    
    print(f"Entrenando con semanas: {train_year_weeks}, testeando en semanas {test_year_weeks}, len train: {len(train_indices)}, len test: {len(test_indices)}, porcentaje: {len(train_indices)/(len(train_indices)+len(test_indices)):.2f}")

    # Comprobar si hay suficientes datos
    if len(train_indices) < 50 or len(test_indices) < 10:
        print(f"Saltando ventana {current_window} por datos insuficientes: train={len(train_indices)}, test={len(test_indices)}")
        continue
    
    # Preparar los datos de entrenamiento y test
    X_train_actual = actual_diferencias.iloc[train_indices].drop(columns=['winnerCode', 'year_week_id'])
    Y_train_actual = actual_diferencias.iloc[train_indices]['winnerCode'].values.reshape(-1, 1)
    
    X_test_actual = actual_diferencias.iloc[test_indices].drop(columns=['winnerCode', 'year_week_id'])
    Y_test_actual = actual_diferencias.iloc[test_indices]['winnerCode'].values.reshape(-1, 1)

    previos_train_index= []
    for idx in train_indices:
        inicio = idx * num_previos
        fin = inicio + num_previos 
        previos_train_index.extend(range(inicio, fin))
    
    previos_test_index= []
    for idx in test_indices:
        inicio = idx * num_previos
        fin = inicio + num_previos  
        previos_test_index.extend(range(inicio, fin))
    
    previos_train_home = previos_home.iloc[previos_train_index].values
    previos_train_away = previos_away.iloc[previos_train_index].values
    previos_test_home = previos_home.iloc[previos_test_index].values
    previos_test_away = previos_away.iloc[previos_test_index].values

    # Reshape para GRU: (n_samples, seq_length, n_features)
    n_train_samples= len(train_indices)
    n_test_samples= len(test_indices)
    seq_length = num_previos
    n_features = input_size_previos

    X_train_previos_home = previos_train_home.reshape(n_train_samples, seq_length, n_features)
    X_train_previos_away = previos_train_away.reshape(n_train_samples, seq_length, n_features)
    X_test_previos_home = previos_test_home.reshape(n_test_samples, seq_length, n_features)
    X_test_previos_away = previos_test_away.reshape(n_test_samples, seq_length, n_features)

    # Convertir a tensores
    tensor_X_train_actual = torch.tensor(X_train_actual.values, dtype=torch.float32).to(device)
    tensor_Y_train_actual = torch.tensor(Y_train_actual, dtype=torch.float32).to(device)
    tensor_X_test_actual = torch.tensor(X_test_actual.values, dtype=torch.float32).to(device)
    tensor_Y_test_actual = torch.tensor(Y_test_actual, dtype=torch.float32).to(device)

    tensor_X_train_previos_home = torch.tensor(X_train_previos_home, dtype=torch.float32).to(device)
    tensor_X_train_previos_away = torch.tensor(X_train_previos_away, dtype=torch.float32).to(device)
    tensor_X_test_previos_home = torch.tensor(X_test_previos_home, dtype=torch.float32).to(device)
    tensor_X_test_previos_away = torch.tensor(X_test_previos_away, dtype=torch.float32).to(device)

    # Early stopping
    best_val_loss = float('inf')
    best_model_state = None
    best_predictions = None
    best_true_values = None
    patience_counter = 0
    epoca_mejor = 0

    # Aplicar warm-up si no es la primera ventana
    if i > 0 and previous_best_model_state is not None:
        model.load_state_dict(previous_best_model_state)
        print(f"Iniciando ventana {current_window} con pesos de la ventana anterior")
    
    # Entrenamiento
    for epoch in range(epochs):
        model.train()
        
        # Forward pass
        y_pred = model(tensor_X_train_actual, tensor_X_train_previos_home, tensor_X_train_previos_away)
        loss = criterion(y_pred, tensor_Y_train_actual)
        
        # Backward pass y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss = loss.item()

        
        model.eval()
        with torch.no_grad():
            y_test_pred = model(tensor_X_test_actual, tensor_X_test_previos_home, tensor_X_test_previos_away)
            test_loss = criterion(y_test_pred, tensor_Y_test_actual).item()

            # Calcular métricas de clasificación
            metrics = calculate_metrics(y_test_pred, tensor_Y_test_actual)

            # Early stopping
            if best_val_loss - test_loss > min_delta:
                best_val_loss = test_loss
                # Guardar el estado del modelo
                best_model_state = copy.deepcopy(model.state_dict())
                patience_counter = 0
                epoca_mejor = epoch
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f'Ventana {current_window}, Época: {epoch}, Train Loss: {train_loss:.8f}, Test Loss: {test_loss:.8f}, AUC: {metrics["auc"]:.4f}, Accuracy: {metrics["accuracy"]:.4f}')
                # Guardar información para visualización
                new_row = pd.DataFrame({
                    'Window': [current_window],
                    'Epoch': [epoch],
                    'TrainingLoss': [train_loss],
                    'ValidationLoss': [test_loss],
                    'Accuracy': [metrics["accuracy"]],
                    'Precision': [metrics["precision"]],
                    'Recall': [metrics["recall"]],
                    'F1': [metrics["f1"]],
                    'AUC': [metrics["auc"]],
                })
                df_visualization_global = pd.concat([df_visualization_global, new_row], ignore_index=True)
                print(f"Early stopping en la ventana {current_window}, época {epoch}")
                break
                

        if epoch % view_step == 0:
            print(f'Ventana {current_window}, Época: {epoch}, Train Loss: {train_loss:.8f}, Test Loss: {test_loss:.8f}, AUC: {metrics["auc"]:.4f}, Accuracy: {metrics["accuracy"]:.4f}')
            


        # Guardar información para visualización
        new_row = pd.DataFrame({
            'Window': [current_window],
            'Epoch': [epoch],
            'TrainingLoss': [train_loss],
            'ValidationLoss': [test_loss],
            'Accuracy': [metrics["accuracy"]],
            'Precision': [metrics["precision"]],
            'Recall': [metrics["recall"]],
            'F1': [metrics["f1"]],
            'AUC': [metrics["auc"]],
        })
        df_visualization_global = pd.concat([df_visualization_global, new_row], ignore_index=True)

    # Cargar el mejor modelo
    if best_model_state is not None:
        model.load_state_dict(best_model_state)  # Cargar el mejor modelo de esta ventana
        previous_best_model_state = copy.deepcopy(best_model_state)  # Guardar para la siguiente ventana
        print(f"Mejor modelo guardado en la época  {epoca_mejor} para la ventana {current_window}")

    print("-------------------") 

# Guardar el modelo
torch.save(model.state_dict(), 'tennis_rnn_model_CLASSIFICATION.pt')

Ventana 1 de 153
Entrenando con semanas: [0 1 2 3 4 5 6 7 8 9], testeando en semanas [10 11 12], len train: 509, len test: 141, porcentaje: 0.78
Ventana 1, Época: 0, Train Loss: 0.69356155, Test Loss: 0.69248962, AUC: 0.5636, Accuracy: 0.5035
Ventana 1, Época: 5, Train Loss: 0.68079734, Test Loss: 0.68827677, AUC: 0.5674, Accuracy: 0.5319
Ventana 1, Época: 10, Train Loss: 0.66094965, Test Loss: 0.68290997, AUC: 0.5803, Accuracy: 0.5603
Ventana 1, Época: 15, Train Loss: 0.64043218, Test Loss: 0.68838543, AUC: 0.5839, Accuracy: 0.5674
Ventana 1, Época: 16, Train Loss: 0.63180083, Test Loss: 0.68885678, AUC: 0.5861, Accuracy: 0.5674
Early stopping en la ventana 1, época 16
Mejor modelo guardado en la época  11 para la ventana 1
-------------------
Ventana 2 de 153
Entrenando con semanas: [ 1  2  3  4  5  6  7  8  9 10], testeando en semanas [11 12 13], len train: 501, len test: 143, porcentaje: 0.78
Iniciando ventana 2 con pesos de la ventana anterior
Ventana 2, Época: 0, Train Loss: 0.65

In [27]:
df_visualization_global.to_csv('resultados/df_visualization_global_CLASSIFICATION.csv', index=False)